# **Distribución de créditos por monto**

In [21]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "vallarta" in nombre and '2019' not in nombre and '2020' not in nombre and '2021' not in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2022_vallarta',
 'df_financiamientos_2023_vallarta',
 'df_financiamientos_2024_vallarta']

#Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

dataframes_procesados = []
for df in dataframes_list:
    dataframes_procesados.append(df)

df_final = pd.concat(dataframes_procesados, ignore_index=True)

In [22]:
df_financiamientos_2022_pachuca = df_financiamientos_2022_vallarta[['id','monto']]
df_financiamientos_2023_pachuca = df_financiamientos_2023_vallarta[['id','monto']]
df_financiamientos_2024_pachuca = df_financiamientos_2024_vallarta[['id','monto']]

## 2022

In [23]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_pachuca['rango'] = df_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_17988\2233432692.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,0.00,1
1,2,1271912.14,2
2,3,310344.57,1
3,4,308885.47,1
4,5,447973.48,1


In [24]:
conteo_por_rango = df_financiamientos_2022_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,810
1,2,242
2,5,96
3,3,92
4,4,34


In [25]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,810
1,2,242
3,3,92
4,4,34
2,5,96


In [26]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    #'1': '< 1M',
    #'2': '< 2M',
    #'3': '< 3M',
    #'4': '< 4M',
    #'5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
   '1': '$0 - 1 millón',
    '2': '$1 - 2 millones',
    '3': '$2 - 3 millones',
    '4': '$3 - 4 millones',
    '5': '$4 - 20 millones'


})
df_conteo_por_rango

,rango,conteo
0,$0 - 1 millón,810
1,$1 - 2 millones,242
3,$2 - 3 millones,92
4,$3 - 4 millones,34
2,$4 - 20 millones,96


In [27]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
#trimestre = tabla_porcentajes['trimestre']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
#trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        #name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
       # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_monto_2022',  carpeta='assets/graficas')
fig.show()

## 2023

In [28]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_pachuca['rango'] = df_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_17988\4015203894.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,2020482.96,3
1,2,697386.25,1
2,3,1753565.84,2
3,4,12918156.00,5
4,5,437683.56,1


In [29]:
conteo_por_rango = df_financiamientos_2023_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,820
1,2,293
2,3,115
3,5,111
4,4,45


In [30]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,820
1,2,293
2,3,115
4,4,45
3,5,111


In [31]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    #'1': '< 1M',
    #'2': '< 2M',
    #'3': '< 3M',
    #'4': '< 4M',
    #'5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
'1': '$0 - 1 millón',
'2': '$1 - 2 millones',
'3': '$2 - 3 millones',
'4': '$3 - 4 millones',
'5': '$4 - 20 millones'

})
df_conteo_por_rango

,rango,conteo
0,$0 - 1 millón,820
1,$1 - 2 millones,293
2,$2 - 3 millones,115
4,$3 - 4 millones,45
3,$4 - 20 millones,111


In [32]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
#trimestre = tabla_porcentajes['trimestre']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
#trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        #name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
      # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_monto_2023',  carpeta='assets/graficas')
fig.show()

## 2024

In [33]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2024_pachuca['rango'] = df_financiamientos_2024_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2024_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_17988\4041216476.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,716650.97,1
1,2,308304.47,1
2,3,7061148.90,5
3,4,0.00,1
4,5,219888.56,1


In [34]:
conteo_por_rango = df_financiamientos_2024_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,640
1,2,267
2,3,126
3,5,104
4,4,43


In [35]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,640
1,2,267
2,3,126
4,4,43
3,5,104


In [36]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    #'1': '< 1M',
    #'2': '< 2M',
    #'3': '< 3M',
    #'4': '< 4M',
    #'5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
    '1': '$0 - 1 millón',
'2': '$1 - 2 millones',
'3': '$2 - 3 millones',
'4': '$3 - 4 millones',
'5': '$4 - 20 millones'

})
df_conteo_por_rango

,rango,conteo
0,$0 - 1 millón,640
1,$1 - 2 millones,267
2,$2 - 3 millones,126
4,$3 - 4 millones,43
3,$4 - 20 millones,104


In [37]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
#trimestre = tabla_porcentajes['trimestre']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
#trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        #name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
       # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_monto_2024',  carpeta='assets/graficas')
fig.show()